In [19]:
pip install langchain

In [20]:
from langchain_huggingface import HuggingFaceEndpoint

In [21]:
# model_id = "tiiuae/falcon-7b-instruct"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
llm_hub = HuggingFaceEndpoint(
    repo_id=model_id,
    temperature=0.1
)

In [22]:
import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
from langchain.embeddings import HuggingFaceInstructEmbeddings
embeddings = HuggingFaceInstructEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": DEVICE}
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [23]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("maths_for_cs.pdf")
documents = loader.load()

# Split the document into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)

# Create an embeddings database using Chroma from the split text chunks.
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embedding=embeddings)

# --> Build the QA chain, which utilizes the LLM and retriever for answering questions. 
# By default, the vectorstore retriever uses similarity search. 
# If the underlying vectorstore support maximum marginal relevance search, you can specify that as the search type (search_type="mmr").
# You can also specify search kwargs like k to use when doing retrieval. k represent how many search results send to llm
from langchain.chains import RetrievalQA
conversation_retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm_hub,
    chain_type="stuff",
    retriever=db.as_retriever(search_type="mmr", search_kwargs={'k': 6, 'lambda_mult': 0.25}),
    return_source_documents=False,
    input_key = "question"
 #   chain_type_kwargs={"prompt": prompt} # if you are using prompt template, you need to uncomment this part
)

In [24]:
# Query the model
global chat_history
output = conversation_retrieval_chain.invoke({"question": "what is the content inside?", "chat_history": ""})
print(output)
answer = output["result"]

{'question': 'what is the content inside?', 'chat_history': '', 'result': ' The content inside is the text of a book or document, which is divided into 14 sections or chapters. The text appears to be about probability theory and statistics, and it includes examples, definitions, and propositions. The content is likely intended for students or professionals in the field of statistics or mathematics. The text does not appear to be a specific book or document, but rather a sample or excerpt from a larger work. The content is written in a formal and technical style, suggesting that it is intended for an audience with a strong background in mathematics and statistics. The text includes mathematical notation and symbols, such as the use of Greek letters and mathematical operators. The content appears to be a mix of theoretical and practical information, with examples and definitions provided to illustrate key concepts. Overall, the content appears to be a sample or excerpt from a textbook or

In [14]:
print(answer)

 The content inside is the text of a book or document, which appears to be a statistics or probability textbook. The text discusses concepts such as partitions, propositions, and events of interest, and includes examples and definitions. It does not appear to be a specific chapter or section title. If you are looking for a specific piece of information, you may need to read the text more closely or search for a specific keyword or phrase.
